In [ ]:
from os import getenv
from pathlib import Path

input_folder = Path(getenv(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input'))
output_folder = Path(getenv(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output'))
output_folder.mkdir(parents=True, exist_ok=True)
datasets_folder = Path('datasets')
archives_folder = datasets_folder / 'archives'

In [ ]:
import json

with (input_folder / 'variables.dictionary').open('rt') as f:
    d = json.load(f)

In [ ]:
from base64 import urlsafe_b64encode
from hashlib import sha256
from os.path import splitext
from urllib.request import urlretrieve as download

from invisibleroads_macros_disk import unarchive_safely

archive_uri = d['uri']
archive_extension = splitext(archive_uri)[1]
archive_name = urlsafe_b64encode(sha256(archive_uri.encode(
    'utf-8')).digest()).decode().rstrip('=') + archive_extension
archive_path = archives_folder / archive_name
archive_folder = Path(splitext(archive_path)[0])
archive_folder

In [ ]:
from os.path import basename

if not archive_folder.exists():
    archive_folder.parent.mkdir(parents=True, exist_ok=True)
    alternate_path = archives_folder / basename(archive_uri)
    if alternate_path.exists():
        archive_path = alternate_path
    else:
        download(archive_uri, archive_path)
    unarchive_safely(str(archive_path), archive_folder)

In [ ]:
from pandas import read_excel

spreadsheet_paths = list(archive_folder.glob('**/*.xlsx'))
if not spreadsheet_paths:
    raise Exception('could not find spreadsheet')
old_table = read_excel(spreadsheet_paths[0], engine='openpyxl')
new_table = old_table[['LON', 'LAT']]
new_table.to_json(output_folder / 'geometries.json', orient='values')